## Real data prediction performance 
Compared with log-ratio methods for comparison (using the same train/test split).  
In each sample, zeros are replaced with $.5x_{\min}$, half the minimum positive value.

We also tested zero replacement by adding a pseudocount of 1, but this approach was overall inferior to $.5x_{\min}$ replacement and is omitted.

In [1]:
import os, sys
os.chdir('..')
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import pandas as pd
import pickle

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', message='Unable to determine R library path')

MASTER_SEED = 20241225

In [2]:
directory = "./datasets/MLRepo/"
x_dir = "/refseq/taxatable.txt"
y_dir = "/task-"  
y_names = {"gevers": "ileum", "ravel": "nugent-score"}

# Ileum microbiome data (binary response)

In [3]:
name = "gevers"
counts = pd.read_csv(directory + name + x_dir, sep='\t')
y_ileum = pd.read_csv(directory + name + y_dir + y_names[name] + ".txt", sep='\t')

counts2 = counts[y_ileum["#SampleID"]]

# Delete taxa present in less than 5 samples
taxa = counts["#OTU ID"][(counts2 > 0).sum(1) >= 5]
counts2 = counts2[(counts2 > 0).sum(1) >= 5]

x = counts2.to_numpy().T
y = y_ileum["Var"].to_numpy().flatten()

# Normalize
x = x / x.sum(1)[:, None]

y_vals = sorted(set(y))
print("Zero ratio:", np.sum(x == 0) / x.size)
print("Data shape", x.shape, " with y =", y_vals[0], "---", np.sum(y==y_vals[0]), "// y =", y_vals[1], "---", np.sum(y==y_vals[1]))

# Y processing (mandatory)
y = np.array([-1. if y == y_vals[0] else 1. for y in y])

Zero ratio: 0.8241899852724595
Data shape (140, 194)  with y = CD --- 78 // y = no --- 62


In [4]:
# Zero replacement and integer conversion of the response (for stratification)
# This is necessary for clr-Kernel (SVM) and clr-RF
x2 = counts2.T.to_numpy().astype(np.float64)
y2 = y.astype(int)
for i in range(x2.shape[0]):
    x2[i] = np.where(x2[i] == 0, x2[i][x2[i] > 0].min() * 0.5, x2[i])

In [5]:
results_df = pd.DataFrame(columns=["CKDR-3", "CKDR-5", "CKDR", "LC-Lasso", "clr-Kernel (SVM)", "clr-RF"])

### CKDR

##### CKDR-3, 5
Target dimension parameter $m$ is fixed to 3 and 5

In [5]:
from functions import parallel_fit_train_test_split

for target_dim in [3, 5]:
    print("\nTarget dimension:", target_dim)
    result = parallel_fit_train_test_split(x, y, type_Y="binary", reps=100, n_jobs=60,
                                           epsilon_list=[0.001, 0.01], dim_list=[target_dim],
                                           sigma_list=np.geomspace(1/2, 2., 5),
                                           save="ileum_dim{}".format(target_dim), 
                                            seed=MASTER_SEED, verbose=False)


Target dimension: 3


[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done   2 out of 100 | elapsed: 15.2min remaining: 746.6min
[Parallel(n_jobs=60)]: Done  23 out of 100 | elapsed: 23.1min remaining: 77.3min
[Parallel(n_jobs=60)]: Done  44 out of 100 | elapsed: 25.3min remaining: 32.2min
[Parallel(n_jobs=60)]: Done  65 out of 100 | elapsed: 32.4min remaining: 17.5min
[Parallel(n_jobs=60)]: Done  86 out of 100 | elapsed: 39.3min remaining:  6.4min


Results of 100 repetitions of train/test split: 0.710357 ± 0.007757

Target dimension: 5


[Parallel(n_jobs=60)]: Done 100 out of 100 | elapsed: 41.1min finished
[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done   2 out of 100 | elapsed: 15.9min remaining: 776.7min
[Parallel(n_jobs=60)]: Done  23 out of 100 | elapsed: 24.9min remaining: 83.5min
[Parallel(n_jobs=60)]: Done  44 out of 100 | elapsed: 27.7min remaining: 35.2min
[Parallel(n_jobs=60)]: Done  65 out of 100 | elapsed: 34.0min remaining: 18.3min
[Parallel(n_jobs=60)]: Done  86 out of 100 | elapsed: 41.9min remaining:  6.8min


Results of 100 repetitions of train/test split: 0.721071 ± 0.007469


[Parallel(n_jobs=60)]: Done 100 out of 100 | elapsed: 44.6min finished


##### CKDR
Dimension grid {3, 4, 5, 6, 7} is included in the cross-validation

In [6]:
result = parallel_fit_train_test_split(x, y, "binary", reps=100, n_jobs=60,
                                           epsilon_list=[0.01, 0.001], dim_list=[3, 4, 5, 6, 7],
                                           sigma_list=np.geomspace(1/2, 2., 5),
                                           save="ileum_dim3-7", verbose=False, seed=MASTER_SEED)

[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done   2 out of 100 | elapsed: 82.7min remaining: 4051.2min
[Parallel(n_jobs=60)]: Done  23 out of 100 | elapsed: 113.7min remaining: 380.7min
[Parallel(n_jobs=60)]: Done  44 out of 100 | elapsed: 134.0min remaining: 170.5min
[Parallel(n_jobs=60)]: Done  65 out of 100 | elapsed: 168.6min remaining: 90.8min
[Parallel(n_jobs=60)]: Done  86 out of 100 | elapsed: 203.3min remaining: 33.1min


Results of 100 repetitions of train/test split: 0.722857 ± 0.007528


[Parallel(n_jobs=60)]: Done 100 out of 100 | elapsed: 211.2min finished


##### Load the saved results

In [6]:
with open("results/realdata-prediction/ileum_dim3.pkl", "rb") as f:
    result = pickle.load(f)
results_df["CKDR-3"] = np.array([res[0] for res in result])

with open("results/realdata-prediction/ileum_dim5.pkl", "rb") as f:
    result = pickle.load(f)
results_df["CKDR-5"] = np.array([res[0] for res in result])

with open("results/realdata-prediction/ileum_dim3-7.pkl", "rb") as f:
    result = pickle.load(f)
results_df["CKDR"] = np.array([res[0] for res in result])

### LC-Lasso
For classification, LC-Lasso implements $\ell_1$-constrained GLM with the zero-sum constraint on the log-transformed compositional data (zeros must be replaced a priori).
- Implementation code is downloaded from: https://github.com/malucalle/CoDA-Penalized-Regression
- Paper: Lu et al. (2019) ["Generalized Linear Models With Linear Constraints for Microbiome Compositional Data"](https://academic.oup.com/biometrics/article/75/1/235/7517229)

In [ ]:
from reproducibility.prediction_comparison import codalasso_clf

# pd.DataFrame-based processing
counts_5min = counts2.copy()
for i in range(counts_5min.shape[1]):
    col = counts_5min.iloc[:, i]
    min_nonzero = col[col > 0].min()
    counts_5min.iloc[:, i] = col.where(col > 0, min_nonzero * 0.5)
counts_5min = counts_5min.T

# LC-Lasso (1min with 100 cores)
result = codalasso_clf(counts_5min, pd.Series(y), cvfolds=5, reps=100,
                       lamseq=np.geomspace(0.001, 1, 30), 
                       njobs=100, seed=MASTER_SEED)
results_df["LC-Lasso"] = result

### clr-Kernel (SVM)
Uses `sklearn.svm.SVC` for SVM fit.  
See the `clr_svm` function in the `prediction_comparison.py` file for details

In [8]:
from reproducibility.prediction_comparison import clr_svm

acc = clr_svm(x2, y2, reps=100, seed=MASTER_SEED)
results_df["clr-Kernel (SVM)"] = acc

Mean accuracy: 0.7167857142857141 Std accuracy: 0.06353831400768181


### clr-RF
Uses `sklearn.ensemble.RandomForestClassifier`.  
See the `clr_rf_clf` function in the `prediction_comparison.py` file for details

In [9]:
from reproducibility.prediction_comparison import clr_rf_clf

acc = clr_rf_clf(x2, y2, reps=100, seed=MASTER_SEED)
results_df["clr-RF"] = acc

Mean accuracy: 0.655 Std accuracy: 0.08809491634432878


### Save the results

In [10]:
# to csv
results_df.to_csv("results/realdata-prediction/ileum_results.csv", index=False)

### Final result

In [12]:
# Load the results from the CSV file
results_df = pd.read_csv("results/realdata-prediction/ileum_results.csv")

# Translate to the misclassification rate
results_df = 1 - results_df

# Calculate mean and standard error for each column
results_df *= 100
means = results_df.mean()
ses = results_df.std() / np.sqrt(len(results_df))

# Create formatted strings with mean (se) up to 3 decimal places
formatted_results = []
for col in results_df.columns:
    formatted_str = f"{means[col]:.1f} ({ses[col]:.1f})"
    formatted_results.append(formatted_str)

# Create new dataframe with same columns but single row containing formatted strings
summary_df = pd.DataFrame([formatted_results], columns=results_df.columns)
summary_df

,CKDR-3,CKDR-5,CKDR,LC-Lasso,clr-Kernel (SVM),clr-RF
0,29.0 (0.8),27.9 (0.8),27.7 (0.8),28.5 (0.7),28.3 (0.6),34.5 (0.9)


In [26]:
# t-test for significance
from scipy.stats import ttest_ind
for col in results_df.columns:
    if col != "CKDR-3":
        t_stat, p_val = ttest_ind(results_df["CKDR-3"], results_df[col], equal_var=False)
        print(f"T-test for CKDR-3 vs {col}: t-statistic = {t_stat:.4f}, p-value = {p_val:.4f}")

T-test for CKDR-3 vs CKDR-5: t-statistic = 0.9900, p-value = 0.3234
T-test for CKDR-3 vs CKDR: t-statistic = 1.1506, p-value = 0.2513
T-test for CKDR-3 vs LC-Lasso: t-statistic = 0.4694, p-value = 0.6393
T-test for CKDR-3 vs clr-Kernel (SVM): t-statistic = 0.6379, p-value = 0.5243
T-test for CKDR-3 vs clr-RF: t-statistic = -4.6924, p-value = 0.0000


# Vaginal microbiome data

In [14]:
name = "ravel"
counts = pd.read_csv(directory + name + x_dir, sep='\t')

Y= pd.read_csv(directory + name + y_dir + y_names[name] + ".txt", sep='\t')

counts2 = counts[Y["#SampleID"]]

# Delete taxa present in less than 5 samples
taxa = counts["#OTU ID"][(counts2 > 0).sum(1) >= 5]
counts2 = counts2[(counts2 > 0).sum(1) >= 5]

X = counts2.to_numpy().T
Y = Y["Var"].to_numpy().flatten()

# Normalize
X = X / X.sum(1)[:, None]

print("Zero ratio:", np.sum(X == 0) / X.size)
print("Data shape", X.shape, " with Y ranges", Y.min(), "to", Y.max())

Zero ratio: 0.9146917910766993
Data shape (388, 241)  with Y ranges 0 to 10


In [15]:
# Zero replacement and integer conversion of the response (for stratification)
# This is necessary for all regression methods
X2 = counts2.T.to_numpy().astype(np.float64)
# Y2 = Y.astype(int)
for i in range(X2.shape[0]):
    X2[i] = np.where(X2[i] == 0, X2[i][X2[i] > 0].min() * 0.5, X2[i])

In [16]:
results_df2 = pd.DataFrame(columns=["CKDR-3", "CKDR-5", "CKDR", "LC-Lasso", "clr-Kernel (KRR)", "clr-RF", "RS-ES"])

### CKDR

##### CKDR-3, 5
Target dimension parameter $m$ is fixed to 3 and 5

In [16]:
from functions import parallel_fit_train_test_split

for target_dim in [3, 5]:
    print("\nTarget dimension:", target_dim)
    result = parallel_fit_train_test_split(X, Y, type_Y=None, reps=100, n_jobs=60,
                                           epsilon_list=[0.01, 0.001], dim_list=[target_dim],
                                           sigma_list=np.geomspace(1/2, 2., 5),
                                           save="vaginal_dim{}".format(target_dim), verbose=False, 
                                           seed=MASTER_SEED)


Target dimension: 3


[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done   2 out of 100 | elapsed: 62.5min remaining: 3064.9min
[Parallel(n_jobs=60)]: Done  23 out of 100 | elapsed: 89.2min remaining: 298.8min
[Parallel(n_jobs=60)]: Done  44 out of 100 | elapsed: 103.8min remaining: 132.1min
[Parallel(n_jobs=60)]: Done  65 out of 100 | elapsed: 127.5min remaining: 68.7min
[Parallel(n_jobs=60)]: Done  86 out of 100 | elapsed: 146.3min remaining: 23.8min
[Parallel(n_jobs=60)]: Done 100 out of 100 | elapsed: 152.1min finished


Results of 100 repetitions of train/test split: 3.392202 ± 0.086100

Target dimension: 5


[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done   2 out of 100 | elapsed: 59.4min remaining: 2908.7min
[Parallel(n_jobs=60)]: Done  23 out of 100 | elapsed: 89.2min remaining: 298.5min
[Parallel(n_jobs=60)]: Done  44 out of 100 | elapsed: 100.5min remaining: 127.9min
[Parallel(n_jobs=60)]: Done  65 out of 100 | elapsed: 120.9min remaining: 65.1min
[Parallel(n_jobs=60)]: Done  86 out of 100 | elapsed: 142.0min remaining: 23.1min
[Parallel(n_jobs=60)]: Done 100 out of 100 | elapsed: 148.0min finished


Results of 100 repetitions of train/test split: 3.367202 ± 0.085657


##### CKDR
Dimension grid {3, 4, 5, 6, 7} is included in the cross-validation

In [17]:
result = parallel_fit_train_test_split(X, Y, type_Y=None, reps=100, n_jobs=60,
                                       epsilon_list=[0.01, 0.001], dim_list=[3, 4, 5, 6, 7],
                                       sigma_list=np.geomspace(1/2, 2., 5),
                                       save="vaginal_dim3-7", verbose=False, seed=MASTER_SEED)

[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done   2 out of 100 | elapsed: 283.1min remaining: 13871.5min
[Parallel(n_jobs=60)]: Done  23 out of 100 | elapsed: 414.0min remaining: 1386.0min
[Parallel(n_jobs=60)]: Done  44 out of 100 | elapsed: 463.4min remaining: 589.8min
[Parallel(n_jobs=60)]: Done  65 out of 100 | elapsed: 597.7min remaining: 321.9min
[Parallel(n_jobs=60)]: Done  86 out of 100 | elapsed: 679.9min remaining: 110.7min
[Parallel(n_jobs=60)]: Done 100 out of 100 | elapsed: 698.0min finished


Results of 100 repetitions of train/test split: 3.411310 ± 0.083639


In [17]:
with open("results/realdata-prediction/vaginal_dim3.pkl", "rb") as f:
    result = pickle.load(f)
results_df2["CKDR-3"] = np.array([res[0] for res in result])

with open("results/realdata-prediction/vaginal_dim5.pkl", "rb") as f:
    result = pickle.load(f)
results_df2["CKDR-5"] = np.array([res[0] for res in result])

with open("results/realdata-prediction/vaginal_dim3-7.pkl", "rb") as f:
    result = pickle.load(f)
results_df2["CKDR"] = np.array([res[0] for res in result])

### LC-Lasso
Implements the log-contrast $\ell_1$-constrained linear regression
- Implementation uses the `c-lasso` library: https://github.com/Leo-Simpson/c-lasso/tree/master
    - You can install it with `pip install c-lasso`.
- Paper: Lin et al. (2014) ["Variable selection in regression with compositional covariates"](https://academic.oup.com/biomet/article-lookup/doi/10.1093/biomet/asu031)

In [18]:
from reproducibility.prediction_comparison import codalasso_reg

# Around 2 minutes with 100 cores
result, _ = codalasso_reg(X2, Y, cvfolds=5, reps=100, 
                       lamseq=np.geomspace(0.001, 1, 30), 
                       njobs=100, seed=MASTER_SEED)
results_df2["LC-Lasso"] = result

[Parallel(n_jobs=100)]: Using backend LokyBackend with 100 concurrent workers.
[Parallel(n_jobs=100)]: Done   6 out of 100 | elapsed:  1.2min remaining: 19.1min
[Parallel(n_jobs=100)]: Done  27 out of 100 | elapsed:  2.1min remaining:  5.6min
[Parallel(n_jobs=100)]: Done  48 out of 100 | elapsed:  2.3min remaining:  2.5min
[Parallel(n_jobs=100)]: Done  69 out of 100 | elapsed:  2.4min remaining:  1.1min
[Parallel(n_jobs=100)]: Done  90 out of 100 | elapsed:  2.6min remaining:   17.0s


Estimated MSE with 100 reps: 3.7645910451964353 +/- 0.6204880350561086


[Parallel(n_jobs=100)]: Done 100 out of 100 | elapsed:  2.9min finished


### clr-Kernel (KRR)
Uses `sklearn.kernel_ridge.KernelRidge` for KRR fit.   
See the `clr_krr` function in the `prediction_comparison.py` file for details

In [19]:
from reproducibility.prediction_comparison import clr_krr

mses = clr_krr(X2, Y, reps=100, seed=MASTER_SEED)
results_df2["clr-Kernel (KRR)"] = mses

Mean MSE: 3.4956057535441043 Std MSE: 0.6363201367562784


### clr-RF
Uses `sklearn.ensemble.RandomForestRegressor`.  
See the `clr_rf_reg` function in the `prediction_comparison.py` file for details

In [20]:
from reproducibility.prediction_comparison import clr_rf_reg

mses = clr_rf_reg(X2, Y, reps=100, seed=MASTER_SEED)
results_df2["clr-RF"] = mses

Mean MSE: 3.3096408589743596 Std MSE: 0.6902166938228487


### RS-ES
Load the matlab-run result  
Implements the relative-shift regression method with equi-sparsity penalty:  
- Li et al. (2023) [It's all relative: Regression analysis with compositional predictors](https://onlinelibrary.wiley.com/doi/full/10.1111/biom.13703)

In [21]:
from scipy.io import loadmat

result = loadmat("./reproducibility/other_methods/Relative-shift/JP_realdata_100reps.mat")["result_test20"].ravel()
print("Mean MSE with 100 reps:", result.mean(), "Std error:", result.std() / np.sqrt(100))
results_df2["RS-ES"] = result

Mean MSE with 100 reps: 4.200485580104152 Std error: 0.08725148989107033


### Save the results

In [22]:
# to csv
results_df2.to_csv("results/realdata-prediction/vaginal_results.csv", index=False)

### Final result

In [23]:
# Load the results from the CSV file
results_df2 = pd.read_csv("results/realdata-prediction/vaginal_results.csv")

means = results_df2.mean()
ses = results_df2.std() / np.sqrt(len(results_df2))

# Create formatted strings with mean (se) up to 3 decimal places
formatted_results = []
for col in results_df2.columns:
    formatted_str = f"{means[col]:.2f} ({ses[col]:.2f})"
    formatted_results.append(formatted_str)

# Create new dataframe with same columns but single row containing formatted strings
summary_df2 = pd.DataFrame([formatted_results], columns=results_df2.columns)
summary_df2

,CKDR-3,CKDR-5,CKDR,LC-Lasso,clr-Kernel (KRR),clr-RF,RS-ES
0,3.39 (0.09),3.37 (0.09),3.41 (0.08),3.76 (0.06),3.50 (0.06),3.31 (0.07),4.20 (0.09)


In [25]:
# t-test for significance
from scipy.stats import ttest_ind
for col in results_df2.columns:
    if col != "CKDR-3":
        t_stat, p_val = ttest_ind(results_df2["CKDR-3"], results_df2[col], equal_var=False)
        print(f"T-test for CKDR-3 vs {col}: t-statistic = {t_stat:.4f}, p-value = {p_val:.4f}")

T-test for CKDR-3 vs CKDR-5: t-statistic = 0.2048, p-value = 0.8379
T-test for CKDR-3 vs CKDR: t-statistic = -0.1584, p-value = 0.8743
T-test for CKDR-3 vs LC-Lasso: t-statistic = -3.4913, p-value = 0.0006
T-test for CKDR-3 vs clr-Kernel (KRR): t-statistic = -0.9610, p-value = 0.3378
T-test for CKDR-3 vs clr-RF: t-statistic = 0.7444, p-value = 0.4575
T-test for CKDR-3 vs RS-ES: t-statistic = -6.5608, p-value = 0.0000
